In [402]:
import os

import dotenv
from tinkoff.invest import Client, GetAssetFundamentalsRequest
from tinkoff.invest.constants import INVEST_GRPC_API_SANDBOX
import pandas as pd

dotenv.load_dotenv('bank.env')
TOKEN = os.getenv('TOKEN')

In [403]:
money_to_invest = int(input())
qual_invest = int(input("1 or 0: "))

with Client(TOKEN, target=INVEST_GRPC_API_SANDBOX) as client:
    shares = client.instruments.shares().instruments
    shares = [i for i in shares if i.currency == "rub"]

In [404]:
TARGET_COLUMNS = [
    "figi", "ticker", "lot", "name",
    "sector", "share_type", "liquidity_flag", "for_qual_investor_flag", "asset_uid", "price"
]
shares_df = pd.DataFrame(columns=TARGET_COLUMNS)
for i, share in enumerate(shares):
    shares_df.loc[i] = pd.Series(
        (
            share.figi,
            share.ticker,
            share.lot,
            share.name,
            share.sector,
            share.share_type,
            share.liquidity_flag,
            share.for_qual_investor_flag,
            share.asset_uid,
            None
        ), index=TARGET_COLUMNS
    )

In [405]:
print(*shares_df["sector"].unique(), sep=', ')

other, it, industrials, materials, telecom, consumer, financial, real_estate, health_care, energy, utilities


In [406]:
fundamentals_columns = [
    "market_capitalization", "high_price_last_52_weeks", "low_price_last_52_weeks",
    "average_daily_volume_last_10_days", "average_daily_volume_last_4_weeks",
    "beta", "free_float", "forward_annual_dividend_yield", "revenue_ttm",
    "ebitda_ttm", "net_income_ttm", "eps_ttm", "diluted_eps_ttm",
    "free_cash_flow_ttm", "five_year_annual_revenue_growth_rate",
    "three_year_annual_revenue_growth_rate", "pe_ratio_ttm", "price_to_sales_ttm",
    "price_to_book_ttm", "price_to_free_cash_flow_ttm", "roe", "roa"
]

TARGET_COLUMNS.extend(fundamentals_columns)
shares_df = pd.DataFrame(columns=TARGET_COLUMNS)

with Client(TOKEN) as client:
    for i, share in enumerate(shares):
        shares_df.loc[i] = pd.Series(
            (
                share.figi,
                share.ticker,
                share.lot,
                share.name,
                share.sector,
                share.share_type,
                share.liquidity_flag,
                share.for_qual_investor_flag,
                share.asset_uid,
                None
            ), index=TARGET_COLUMNS[:10]
        )
        try:
            request = GetAssetFundamentalsRequest(assets=[share.asset_uid])
            fundamentals_data = client.instruments.get_asset_fundamentals(request=request).fundamentals[0]
        except:
            print(i, share.name, "UID not found")
            fundamentals_data = None

        for col in fundamentals_columns:
            shares_df.at[i, col] = getattr(fundamentals_data, col, None)

    for i, row in shares_df.iterrows():
        if row.for_qual_investor_flag and not qual_invest:
            shares_df.at[i, "price"] = float('inf')
        else:
            last_price_data = client.market_data.get_last_prices(figi=[row.figi]).last_prices[0]
            price = last_price_data.price.units + last_price_data.price.nano / 10e9
            shares_df.at[i, "price"] = price * row.lot

17b7739dd75f99a3f12c5081259ff27d GetAssetFundamentals NOT_FOUND 50009
61bbe3c2b38840ed1983e30431be40bc GetAssetFundamentals NOT_FOUND 50009


46 Озон Фармацевтика UID not found
50 Обьнефтегазгеология - ао UID not found


a7b426c1c91d3ff0b2fbea8feff62573 GetAssetFundamentals NOT_FOUND 50009
398b22299604ba7632723b99ffe1b807 GetAssetFundamentals NOT_FOUND 50009


87 Обьнефтегазгеология - привилегированные акции UID not found
88 Уфаоргсинтез - привилегированные акции UID not found


c5a6287ad08fd1cf52e78d14d193a7f7 GetAssetFundamentals NOT_FOUND 50009
ece3734a6ecba25804f8f6e91d33c4af GetAssetFundamentals NOT_FOUND 50009


104 VEON UID not found
105 НИТЕЛ UID not found


In [407]:
display(shares_df)

,figi,ticker,lot,name,sector,share_type,liquidity_flag,for_qual_investor_flag,asset_uid,price,...,diluted_eps_ttm,free_cash_flow_ttm,five_year_annual_revenue_growth_rate,three_year_annual_revenue_growth_rate,pe_ratio_ttm,price_to_sales_ttm,price_to_book_ttm,price_to_free_cash_flow_ttm,roe,roa
0,TCS007940839,KZIZP,1,Красногорский завод им. С.А. Зверева - привиле...,other,2,False,True,1d1f0048-25a4-4a77-ace7-693fb74a0fa6,inf,...,0.0,-865456000.0,0.0,2.47,2.88,0.02,0.04,-0.48,1.52,0.32
1,TCS00A108ZR8,DATA,1,Группа Аренадата,it,1,True,False,4cda1996-5fa5-4c56-9442-958e2eb20afc,121.018,...,0.0,0.0,0.0,0.0,0.0,0.0,24.65,0.0,0.0,0.0
2,BBG000FWGSZ5,IRKT,100,Яковлев,industrials,1,True,False,6faba6f9-1ba7-4c3b-92ad-a0ec9ddf1303,2809.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.59,0.0,-21.41,-2.76
3,BBG004S68CV8,VSMO,1,ВСМПО-АВИСМА,materials,1,True,False,f712e646-7c5d-4266-aabd-a18c9c65a7e2,23320.0,...,0.0,-2986306000.0,0.0,10.91,10.69,2.1,0.85,-89.11,8.29,4.82
4,BBG000Q7ZZY2,UNAC,1000,Объединенная авиастроительная корпорация,industrials,1,True,False,68af184b-07c1-4f27-8889-62d869f9dc0e,69.55,...,0.0,21392000000.0,0.0,3.33,0.0,1.34,10.97,31.61,-68.84,-3.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,BBG004S68BR5,NMTP,100,НМТП,industrials,1,True,False,84cd22cb-6bbf-42cc-944c-e57af772d3d5,803.3,...,0.0,0.0,0.0,13.96,4.69,2.25,1.04,0.0,23.58,17.17
168,BBG000QFH687,TGKA,100000,ТГК-1,utilities,1,True,False,8558bad9-44a4-4f91-bca7-9c6e8bcd454f,67.8,...,0.0,0.0,0.0,3.42,0.0,0.0,0.17,0.0,4.96,3.79
169,BBG0027F0Y27,CNTLP,100,Центральный Телеграф - акции привилегированные,telecom,2,False,False,2dd88e4e-0531-41e5-8c41-14625ad84240,707.4,...,0.0,146024000.0,0.0,-7.13,62.01,0.33,0.52,2.88,0.82,0.57
170,BBG004S68FR6,MTLRP,10,Мечел - Привилегированные акции,materials,2,True,False,cb1c065d-9d85-4aeb-9e31-41fafb7db8e1,1000.0,...,0.0,28711000000.0,0.0,15.2,1.17,0.03,-0.19,0.48,-16.11,4.36


In [408]:
def analyze_and_select_portfolio(budget, is_qualified_investor=False):
    def calculate_scores(row):

        if row["market_capitalization"] is None:
            return 0

        valuation_score = (
                                  (1 / row['pe_ratio_ttm'] if row['pe_ratio_ttm'] > 0 else 0) * 0.4 +
                                  (1 / row['price_to_sales_ttm'] if row['price_to_sales_ttm'] > 0 else 0) * 0.3 +
                                  (1 / row['price_to_book_ttm'] if row['price_to_book_ttm'] > 0 else 0) * 0.3
                          ) * 0.3

        growth_score = (
                               row['three_year_annual_revenue_growth_rate'] * 0.5 +
                               row['five_year_annual_revenue_growth_rate'] * 0.3 +
                               row['eps_ttm'] * 0.2
                       ) * 0.2

        profitability_score = (
                                      row['roe'] * 0.5 +
                                      row['roa'] * 0.3 +
                                      (row['net_income_ttm'] / row['market_capitalization'] if row[
                                                                                                   'market_capitalization'] > 0 else 0) * 0.2
                              ) * 0.2

        liquidity_score = (
                                  row['market_capitalization'] * 0.4 +
                                  row['average_daily_volume_last_10_days'] * 0.4 +
                                  row['free_float'] * 0.2
                          ) * 0.15

        risk_score = (
                             (1 / row['beta'] if row['beta'] > 0 else 0) * 0.5 +
                             ((row['high_price_last_52_weeks'] - row['low_price_last_52_weeks']) / row[
                                 'high_price_last_52_weeks'] if row['high_price_last_52_weeks'] > 0 else 0) * 0.5
                     ) * 0.15

        return (valuation_score + growth_score + profitability_score + liquidity_score + risk_score) / 1e9

    shares_df["investment_score"] = shares_df.apply(calculate_scores, axis=1)

    def knapsack_optimization(items, max_weight):
        n = len(items)
        dp = [[0] * (max_weight + 1) for _ in range(n + 1)]

        for i in range(1, n + 1):
            score = items[i - 1]["investment_score"]
            weight = int(items[i - 1]["price"])

            for w in range(max_weight + 1):
                if weight <= w:
                    dp[i][w] = max(dp[i - 1][w], dp[i - 1][w - weight] + score)
                else:
                    dp[i][w] = dp[i - 1][w]

        w = max_weight
        selected_items = []
        for i in range(n, 0, -1):
            if dp[i][w] != dp[i - 1][w]:
                selected_items.append(items[i - 1])
                w -= int(items[i - 1]["price"])

        return selected_items

    shares_to_consider = shares_df[shares_df["price"] < float('inf')][["figi", "ticker", "investment_score", "price"]].to_dict("records")
    optimal_portfolio = knapsack_optimization(shares_to_consider, budget)

    return pd.DataFrame(optimal_portfolio)

In [409]:
s = 0
portfolio = analyze_and_select_portfolio(money_to_invest, qual_invest)
for i, row in portfolio.iterrows():
    s += row.iloc[3]
print(f"Money to spend on portfolio {round(s, 6)}, money remains: {round(money_to_invest - s, 6)}")
display(portfolio)

Money to spend on portfolio 9403.64, money remains: -3.64


,figi,ticker,investment_score,price
0,BBG000VJMH65,MRKS,3.038202,54.550
1,BBG004731489,GMKN,91.481204,980.560
2,BBG00475K2X9,HYDR,13.410712,50.510
3,BBG000K3STR7,APTK,4.763437,100.452
4,BBG004S684M6,SIBN,163.873630,574.050
5,BBG004730RP0,GAZP,178.306914,1250.600
6,BBG004730N88,SBER,315.348956,2440.240
7,BBG00ZHCX1X2,FIXP,8.670009,167.070
8,BBG004731354,ROSN,282.335744,444.025
9,BBG00475KKY8,NVTK,157.657298,871.040
